# Build a Retrieval Augmented Generation (RAG) App

![RAG Indexing](https://python.langchain.com/assets/images/rag_indexing-8160f90a90a33253d0154659cf7d453f.png)

# Installation 
This tutorial requires the `langchain` dependency.

In [1]:
%%capture
%pip install --quiet --upgrade langchain langchain-community langchain-chroma

In [2]:
%%capture
%pip install -qU langchain-openai

# Data Preprocessing Workflow

In [3]:
transformer = {
    "transformer_id":1,
    "Customer": "Honda Automobile (Thailand) Co., Ltd.",
    "Location": "Ayutthaya Sub.1",
    "Technical Data": {
        "Manufacture": "DAIHEN",
        "Type": "PP0318B02",
        "Rated voltage": "115",
        "Rated frequency": "50",
        "Rated power": "20000/25000"
    },
    "On-Load Tap Changer": {
        "Manufacture": "MR",
        "Type": "Vlll 200D-123/76-1019 3W",
        "Rated voltage": "400/230",
        "Rated current": "161"
    }
}

In [4]:
import pandas as pd
from sqlalchemy import create_engine

# Path to your Excel file
file_path = 'data/data_poc.xlsx'

# SQLite database setup
engine = create_engine('sqlite:///transformer_data.db')

# Load the Excel file
xls = pd.ExcelFile(file_path)

# Process each sheet and save to SQLite
for sheet_name in xls.sheet_names:
    # Load each sheet into a DataFrame
    df = pd.read_excel(file_path, sheet_name=sheet_name)
    
    # Add transformer_id column
    df['transformer_id'] = 1

    # Clean the DataFrame (remove completely empty rows/columns)
    df = df.dropna(how='all', axis=0)  # Remove empty rows
    df = df.dropna(how='all', axis=1)  # Remove empty columns

    # Save the DataFrame to SQLite
    table_name = sheet_name.replace(" ", "_").lower()  # Normalize table name
    df.to_sql(table_name, engine, index=False, if_exists='replace')

print("Data from all sheets has been successfully saved to the SQLite database.")


Data from all sheets has been successfully saved to the SQLite database.


In [5]:
import sqlite3

# Path to your SQLite database
db_path = 'transformer_data.db'

# Connect to the database
conn = sqlite3.connect(db_path)

# Query to list all tables
query = "SELECT name FROM sqlite_master WHERE type='table';"
cursor = conn.cursor()
cursor.execute(query)

# Fetch all table names
tables = cursor.fetchall()

# Print table names
print("Tables in the database:")
for table in tables:
    print(table[0])

# Close the connection
conn.close()


Tables in the database:
t1
t2
t3
t4
t5
t6
t7
t8
t9
t10
t11
t12


In [4]:
import os
import sqlite3
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate

# Set up OpenAI API key
# os.environ["OPENAI_API_KEY"] = "your_openai_api_key"  # Replace with your OpenAI API Key

# Connect to the SQLite database
db_path = "transformer_data.db"  # Path to your SQLite database
conn = sqlite3.connect(db_path)

# Function to load data from SQLite and create documents
def load_transformer_data(transformer_id):
    query = f"SELECT * FROM t1 WHERE transformer_id = {transformer_id}"  # Adjust table name as needed
    df = pd.read_sql_query(query, conn)

    # Combine rows into a single document with context
    transformer_docs = []
    for _, row in df.iterrows():
        doc_content = f"""
        Transformer ID: {transformer_id}
        Year: {row['year']}
        HV to GND: {row.get('HV to GND', 'N/A')}
        LV to GND: {row.get('LV to GND', 'N/A')}
        HV to LV: {row.get('HV to LV', 'N/A')}
        Measuring Type: {row.get('measuring_type', 'N/A')}
        """
        transformer_docs.append(doc_content)
    return transformer_docs

# Load data for transformer_id = 1
transformer_id = 1
docs = load_transformer_data(transformer_id)

# Split documents into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_text("\n".join(docs))

# Generate embeddings and store in Chroma
embeddings = OpenAIEmbeddings()
vectorstore = Chroma.from_texts(splits, embeddings)

# Set up the retriever
retriever = vectorstore.as_retriever()

# Define the RAG pipeline
llm = OpenAI(temperature=0, model="gpt-4o")
prompt_template = """
You are a transformer maintenance assistant. Based on the following transformer data:

{context}

Answer the following question:
{question}
"""
prompt = PromptTemplate(template=prompt_template, input_variables=["context", "question"])

qa_chain = RetrievalQA(llm=llm, retriever=retriever, return_source_documents=True, prompt=prompt)

# Query the RAG system
question = "What was the HV to GND value for 2561?"
response = qa_chain.run(question)

# Display the response
print("Response:")
print(response)


/tmp/ipykernel_3976/1322626330.py:45: LangChainDeprecationWarning:

The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.



ValidationError: 1 validation error for OpenAIEmbeddings
  Value error, Did not find openai_api_key, please add an environment variable `OPENAI_API_KEY` which contains it, or pass `openai_api_key` as a named parameter. [type=value_error, input_value={'model_kwargs': {}, 'cli...20, 'http_client': None}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.9/v/value_error

In [ ]:
import getpass
import os
import bs4
os.environ["OPENAI_API_KEY"] = os.environ.get("OPENAI_API_KEY")
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o-mini")
# Load, chunk and index the contents of the blog.
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

rag_chain.invoke("What is Task Decomposition?")

ImportError: cannot import name 'LangSmithParams' from 'langchain_core.language_models.chat_models' (/usr/local/lib/python3.10/dist-packages/langchain_core/language_models/chat_models.py)

In [15]:
# cleanup
vectorstore.delete_collection()